In [2]:
import shutil
#폴더삭제
shutil.rmtree("./openeyes")
shutil.rmtree("./closeeyes") 

In [ ]:
# 라이브러리 설치
!pip install tensorflow opencv-python cvlib

In [1]:
import os 
# 폴더 생성
os.mkdir("./mask")
os.mkdir("./nomask") 

In [29]:
# 데이터 수집
import cv2
import cvlib as cv
 
# open webcam (웹캠 열기)
webcam = cv2.VideoCapture(0)
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
 
sample_num = 0    
captured_num = 0
    
# loop through frames
while webcam.isOpened():
    
    # read frame from webcam 
    status, frame = webcam.read()
    sample_num = sample_num + 1
    
    if not status:
        break
 
    # 이미지 내 얼굴 검출
    face, confidence = cv.detect_face(frame)
    
    print(face)
    print(confidence)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
 
 
        if sample_num % 8  == 0:
            captured_num = captured_num + 1
            face_in_img = frame[startY:endY, startX:endX, :]
            #cv2.imwrite('./mask/face'+str(captured_num)+'.jpg', face_in_img) # 마스크 착용 데이터 수집시 주석해제
            cv2.imwrite('./nomask/face'+str(captured_num)+'.jpg', face_in_img) # 마스크 미착용 데이터 수집시 주석해제
            
    # display output
    cv2.imshow("captured frames", frame)        
    
    # 수집 완료 시 키보드 'q'키를 눌러서 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows()   


[[103, 333, 274, 523]]
[0.93214357]
[[109, 333, 267, 537], [855, 215, 1100, 546]]
[0.9992036, 0.7470435]
[[108, 334, 268, 538], [852, 221, 1102, 547]]
[0.99925023, 0.65836614]
[[108, 334, 267, 539], [852, 222, 1101, 545]]
[0.9992907, 0.62216073]
[[109, 333, 267, 541], [851, 224, 1101, 550]]
[0.9996119, 0.66121686]
[[109, 333, 267, 541], [850, 225, 1100, 550]]
[0.999635, 0.67911947]
[[109, 333, 267, 542], [850, 224, 1099, 555]]
[0.9996068, 0.7052217]
[[109, 333, 267, 541], [850, 223, 1099, 555]]
[0.9995235, 0.72701794]
[[109, 333, 267, 541], [850, 223, 1100, 555]]
[0.99955994, 0.73499304]
[[109, 333, 267, 541], [850, 223, 1100, 556]]
[0.99961543, 0.74494547]
[[109, 333, 267, 541], [850, 223, 1099, 556]]
[0.99952376, 0.74536544]
[[109, 333, 267, 541], [850, 223, 1099, 559]]
[0.9995078, 0.75085074]
[[109, 333, 267, 541], [850, 223, 1099, 558]]
[0.99948066, 0.7720877]
[[109, 333, 267, 541], [849, 223, 1099, 560]]
[0.99957746, 0.7588656]
[[109, 333, 267, 541], [850, 223, 1099, 559]]
[0.9994

[[110, 334, 268, 542], [847, 220, 1091, 578]]
[0.9996401, 0.81372374]
[[109, 334, 266, 543], [849, 222, 1096, 581]]
[0.9996338, 0.82228404]
[[110, 336, 267, 546], [853, 224, 1100, 576]]
[0.99962413, 0.79736876]
[[110, 335, 266, 543], [854, 223, 1101, 575]]
[0.9995467, 0.8407746]
[[110, 334, 267, 542], [854, 224, 1100, 577]]
[0.99944884, 0.8220132]
[[110, 334, 267, 542], [854, 222, 1099, 577]]
[0.9993538, 0.8424674]
[[110, 334, 267, 541], [854, 220, 1098, 571]]
[0.9996082, 0.8123036]
[[110, 334, 265, 543], [853, 221, 1098, 570]]
[0.9996113, 0.79902834]
[[110, 333, 267, 542], [852, 219, 1100, 567]]
[0.99951684, 0.8305906]
[[111, 332, 268, 542], [853, 219, 1103, 565]]
[0.99937844, 0.86777586]
[[110, 331, 267, 540], [855, 217, 1107, 569]]
[0.9994142, 0.9011174]
[[110, 333, 267, 542], [857, 221, 1108, 572]]
[0.9993605, 0.90457726]
[[110, 334, 267, 541], [857, 221, 1109, 570]]
[0.9994234, 0.9100528]
[[111, 334, 267, 542], [857, 223, 1107, 567]]
[0.9995209, 0.89007616]
[[109, 333, 267, 542], 

[[112, 333, 268, 542], [859, 242, 1117, 570]]
[0.99890614, 0.93569523]
[[112, 333, 268, 541], [859, 242, 1115, 566]]
[0.9987651, 0.92046815]
[[112, 333, 268, 541], [859, 241, 1114, 565]]
[0.9987596, 0.9070631]
[[112, 333, 267, 542], [858, 240, 1112, 565]]
[0.99888664, 0.9086903]
[[112, 334, 268, 542], [858, 241, 1113, 564]]
[0.99916613, 0.9142917]
[[112, 333, 268, 542], [858, 241, 1113, 565]]
[0.99924904, 0.9189057]
[[112, 333, 268, 542], [858, 241, 1113, 569]]
[0.99924386, 0.93887764]
[[113, 334, 268, 542], [859, 241, 1113, 566]]
[0.99915695, 0.9221558]
[[112, 334, 268, 543], [858, 242, 1115, 568]]
[0.9991736, 0.9309476]
[[112, 334, 268, 543], [859, 242, 1115, 567]]
[0.99921227, 0.90764564]
[[112, 334, 268, 542], [859, 242, 1114, 567]]
[0.9990771, 0.90931106]
[[112, 333, 267, 542], [860, 241, 1111, 569]]
[0.9990707, 0.8859484]
[[112, 333, 267, 542], [858, 239, 1106, 569]]
[0.99896896, 0.82943267]
[[112, 334, 268, 542], [858, 239, 1105, 572]]
[0.99906176, 0.8186459]
[[112, 334, 267, 54

[[113, 336, 268, 543], [856, 247, 1115, 565]]
[0.9993162, 0.8776755]
[[113, 335, 268, 543], [855, 247, 1115, 566]]
[0.99908996, 0.8692595]
[[113, 335, 267, 541], [856, 247, 1115, 562]]
[0.9988003, 0.8637502]
[[113, 335, 266, 542], [856, 247, 1115, 560]]
[0.9989619, 0.83764696]
[[112, 335, 267, 542], [856, 247, 1115, 559]]
[0.9992803, 0.8356679]
[[113, 335, 266, 542], [856, 247, 1115, 558]]
[0.9990615, 0.8418671]
[[112, 335, 267, 542], [856, 247, 1114, 564]]
[0.9990815, 0.8739411]
[[113, 335, 266, 542], [856, 246, 1113, 563]]
[0.9988991, 0.8576355]
[[113, 335, 267, 542], [856, 247, 1112, 558]]
[0.99911934, 0.79816264]
[[113, 335, 267, 542], [854, 246, 1110, 562]]
[0.99862874, 0.828474]
[[113, 335, 266, 541], [853, 245, 1109, 558]]
[0.9988324, 0.7757158]
[[113, 335, 267, 542], [853, 245, 1108, 558]]
[0.9985979, 0.7811022]
[[113, 335, 266, 541], [853, 245, 1107, 560]]
[0.9984743, 0.80304456]
[[113, 335, 267, 541], [853, 245, 1107, 562]]
[0.9981122, 0.81980413]
[[113, 335, 267, 542], [853,

[[111, 336, 268, 541], [806, 237, 1047, 556]]
[0.9958633, 0.61287]
[[113, 336, 267, 540], [820, 236, 1065, 565]]
[0.99818724, 0.5393331]
[[112, 332, 268, 540], [826, 231, 1078, 570]]
[0.99937624, 0.5142096]
[[112, 333, 269, 539], [838, 224, 1084, 577]]
[0.9993801, 0.5941926]
[[110, 335, 267, 542], [842, 224, 1087, 569]]
[0.99932325, 0.6491933]
[[111, 339, 266, 543], [844, 225, 1088, 568]]
[0.9987556, 0.61570954]
[[112, 338, 266, 543], [844, 225, 1087, 578]]
[0.99852353, 0.7126877]
[[110, 334, 268, 541], [844, 227, 1088, 579]]
[0.9995473, 0.7010522]
[[113, 333, 268, 540], [843, 227, 1088, 579]]
[0.9991684, 0.7154778]
[[112, 334, 268, 540], [843, 227, 1086, 587]]
[0.9993667, 0.7137042]
[[112, 336, 266, 542], [844, 229, 1084, 579]]
[0.9988914, 0.63358474]
[[113, 338, 266, 543], [842, 229, 1082, 587]]
[0.99908495, 0.6959391]
[[113, 337, 265, 544], [842, 232, 1083, 585]]
[0.9990471, 0.63167644]
[[113, 336, 266, 541], [841, 234, 1082, 589]]
[0.9993211, 0.6416047]
[[114, 333, 268, 539], [840,

[[109, 338, 266, 543], [808, 245, 1042, 512]]
[0.9997118, 0.7642104]
[[109, 338, 266, 544], [811, 246, 1047, 508]]
[0.99968445, 0.78870124]
[[109, 339, 267, 544], [818, 246, 1056, 513]]
[0.9997271, 0.7751288]
[[110, 338, 266, 544], [822, 245, 1062, 507]]
[0.9996575, 0.80558723]
[[109, 338, 267, 544], [826, 244, 1063, 513]]
[0.9996574, 0.723327]
[[110, 338, 266, 543], [822, 243, 1068, 517]]
[0.9996408, 0.7555746]
[[110, 338, 266, 544], [824, 243, 1070, 518]]
[0.9996221, 0.7533236]
[[110, 337, 266, 543], [824, 242, 1073, 523]]
[0.99959916, 0.70076925]
[[110, 337, 266, 544], [824, 240, 1075, 525]]
[0.9996916, 0.65889657]
[[110, 337, 266, 543], [827, 240, 1075, 523]]
[0.9995937, 0.66774064]
[[110, 337, 265, 544], [829, 241, 1076, 521]]
[0.99961025, 0.66825193]
[[110, 337, 266, 543], [830, 241, 1077, 519]]
[0.99953556, 0.67020476]
[[109, 338, 266, 544], [828, 240, 1077, 522]]
[0.9996377, 0.6485973]
[[109, 339, 264, 543], [828, 240, 1077, 523]]
[0.99955505, 0.63442564]
[[108, 339, 265, 543],

In [27]:
webcam.release()
cv2.destroyAllWindows()   


In [34]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
 
path_dir1 = './mask/'
path_dir2 = './nomask/'
path_dir3 = './openeyes/'
path_dir4 = './closeeyes/'
path_dir5 = './up/'
 
file_list1 = [file for file in os.listdir(path_dir1) if file.endswith('.jpg')] # path에 존재하는 파일 목록 가져오기
file_list2 = [file for file in os.listdir(path_dir2) if file.endswith('.jpg')]
#file_list3 = [file for file in os.listdir(path_dir3) if file.endswith('.jpg')]
#file_list4 = [file for file in os.listdir(path_dir4) if file.endswith('.jpg')]
#file_list5 = [file for file in os.listdir(path_dir5) if file.endswith('.jpg')]
 
file_list1_num = len(file_list1)
file_list2_num = len(file_list2)
#file_list3_num = len(file_list3)
#file_list4_num = len(file_list4)
#file_list5_num = len(file_list5)
 
file_num = file_list1_num + file_list2_num# + file_list3_num + file_list4_num + file_list5_num
 
 
#%% 이미지 전처리
num = 0;
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) 
all_label = np.float64(np.zeros((file_num, 1)))
 
for img_name in file_list1:
    img_path = path_dir1+img_name
    print(num)
    print(img_path)
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1
 
for img_name in file_list2:
    img_path = path_dir2+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1
""" 
for img_name in file_list3:
    img_path = path_dir3+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 2 # openeyes
    num = num + 1
 
for img_name in file_list4:
    img_path = path_dir4+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 3 # closeeyes
    num = num + 1

for img_name in file_list5:
    img_path = path_dir5+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 4 # up
    num = num + 1
""" 
 
# 데이터셋 섞기(적절하게 훈련되게 하기 위함) 
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)
 
all_label = all_label[indices]
all_img = all_img[indices]
 
 
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))
 
train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 
 
train_label = all_label[0:num_train]
test_label = all_label[num_train:]
 
 
 
#%% 
# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)
 
base_model = ResNet50(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False
base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))
 
flatten_layer = Flatten()
dense_layer1 = Dense(128, activation='relu')
bn_layer1 = BatchNormalization()
dense_layer2 = Dense(1, activation=tf.nn.sigmoid)
 
model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        bn_layer1,
        dense_layer2,
        ])
 
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
 
model.fit(train_img, train_label, epochs=10, batch_size=16, validation_data = (test_img, test_label))
 
 
# save model
model.save("model.h5")
 
print("Saved model to disk")  


0
./mask/face11.jpg
1
./mask/face39.jpg
2
./mask/face172.jpg
3
./mask/face166.jpg
4
./mask/face199.jpg
5
./mask/face204.jpg
6
./mask/face210.jpg
7
./mask/face238.jpg
8
./mask/face239.jpg
9
./mask/face211.jpg
10
./mask/face205.jpg
11
./mask/face198.jpg
12
./mask/face167.jpg
13
./mask/face173.jpg
14
./mask/face38.jpg
15
./mask/face10.jpg
16
./mask/face12.jpg
17
./mask/face165.jpg
18
./mask/face171.jpg
19
./mask/face159.jpg
20
./mask/face213.jpg
21
./mask/face207.jpg
22
./mask/face206.jpg
23
./mask/face212.jpg
24
./mask/face158.jpg
25
./mask/face170.jpg
26
./mask/face164.jpg
27
./mask/face13.jpg
28
./mask/face17.jpg
29
./mask/face148.jpg
30
./mask/face160.jpg
31
./mask/face174.jpg
32
./mask/face216.jpg
33
./mask/face202.jpg
34
./mask/face203.jpg
35
./mask/face217.jpg
36
./mask/face175.jpg
37
./mask/face161.jpg
38
./mask/face149.jpg
39
./mask/face16.jpg
40
./mask/face28.jpg
41
./mask/face349.jpg
42
./mask/face14.jpg
43
./mask/face177.jpg
44
./mask/face163.jpg
45
./mask/face188.jpg
46
./mas

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 12,845,569
Non-trainable params: 23,587,968
_________________________________________________________________
Epoch 1/10
29/29 [==============================] - 27s 871ms/step - loss: 0.0400 - accuracy: 0.980

In [29]:
!pip install pyttsx3

     |████████████████████████████████| 273 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 1.4 MB/s eta 0:00:01


     |████████████████████████████████| 293 kB 1.4 MB/s eta 0:00:01


In [9]:
!pip freeze | grep -E '(Pillow|PIL)'

Pillow==8.2.0


In [33]:
# import necessary packages
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image

 
model = load_model('model.h5')
model.summary()
 
# open webcam
webcam = cv2.VideoCapture(0)
 
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()

    
# loop through frames
while webcam.isOpened():
    # read frame from webcam 
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    prediction = 1
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
            #predicted_class = np.round(prediction[0]) # 예측된 클래스 0, 1, 2
            #print(prediction[0])
            #print(predicted_class)
            
            if prediction < 0.5: # 마스크 미착용으로 판별되면, 
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                
            else: # 마스크 착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
         
    # display output
    cv2.imshow("mask nomask classify", frame)
 
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 12,845,569
Non-trainable params: 23,587,968
_________________________________________________________________


In [21]:
np.argmax(2.21321)

0

In [26]:
webcam.release()
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1